In [0]:
bronze_output = dbutils.jobs.taskValues.get(taskKey = "Bronze", key = "bronze_output")

start_date = bronze_output.get("start_date","")
bronze_adls = bronze_output.get("bronze_adls","")
silver_adls = bronze_output.get("silver_adls","")

print(f"Start date: {start_date}, Silver adls: {silver_adls}")

In [0]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")


In [0]:
df.head()

Row(geometry=Row(coordinates=[-151.6239, 61.7109, 83.4], type='Point'), id='ak025dfawa3s', properties=Row(alert=None, cdi=None, code='025dfawa3s', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak025dfawa3s&format=geojson', dmin=None, felt=None, gap=None, ids=',ak025dfawa3s,', mag=1.9, magType='ml', mmi=None, net='ak', nst=None, place='33 km SSW of Skwentna, Alaska', rms=0.55, sig=56, sources=',ak,', status='automatic', time=1760917613181, title='M 1.9 - 33 km SSW of Skwentna, Alaska', tsunami=0, type='earthquake', types=',origin,phase-data,', tz=None, updated=1760917820515, url='https://earthquake.usgs.gov/earthquakes/eventpage/ak025dfawa3s'), type='Feature')

In [0]:
# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [0]:
df.head()

Row(id='ak025dfawa3s', longitude=-151.6239, latitude=61.7109, elevation=83.4, title='M 1.9 - 33 km SSW of Skwentna, Alaska', place_description='33 km SSW of Skwentna, Alaska', sig=56, mag=1.9, magType='ml', time=1760917613181, updated=1760917820515)

In [0]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)



In [0]:

# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)

In [0]:
df.head()

Row(id='ak025dfawa3s', longitude=-151.6239, latitude=61.7109, elevation=83.4, title='M 1.9 - 33 km SSW of Skwentna, Alaska', place_description='33 km SSW of Skwentna, Alaska', sig=56, mag=1.9, magType='ml', time=datetime.datetime(2025, 10, 19, 23, 46, 53, 181000), updated=datetime.datetime(2025, 10, 19, 23, 50, 20, 515000))

In [0]:
silver_output_path = f"{silver_adls}earthquake_events_silver/"

In [0]:
df.write.mode('append').parquet(silver_output_path)

In [0]:
dbutils.jobs.taskValues.set(key = "silver_output",value = silver_output_path)